In [ ]:
import pandas as pd
import requests
import re

In [ ]:
API_URL = "https://api-inference.huggingface.co/models/mistralai/Mistral-7B-Instruct-v0.3"
headers = {"Authorization": "Bearer "} 

In [ ]:
file_path = r"sample-dataset-1000-reddit-posts.csv"
df = pd.read_csv(file_path)

In [ ]:
df['selftext'].fillna('', inplace=True)

cols_to_drop = ['url', 'url_overridden_by_dest', 'is_gallery',
       'is_video', 'archived', 'permalink', 'author',
       'author_fullname', 'name', 'thumbnail', 'media', 'preview',
       'first_image_url', 'gallery_data', 'media_metadata', 'subreddit',
       'subreddit_id', 'subreddit_subscribers',
       'subreddit_type', 'num_comments', 'total_awards_received',
       'upvote_ratio', 'ups', 'score', 'total_score']

df.drop(cols_to_drop, axis=1, inplace=True)

In [ ]:
df['text'] = 'Post title: ' + df['title'] + '. Post description: ' + df['selftext']

In [ ]:
def query(payload):
	response = requests.post(API_URL, headers=headers, json=payload)
	return response.json()

In [ ]:
def reddit_classifier(input_text):
    
    prompt2 = f"""Classify the input text given at the end, into 1 of the 3 given categories (output the correct number only- 1,2 or 3), based on the text content. 
                    The categories: 
                    Category 1. Input Text content mentions figuring out if an image is AI-generated or not 
                    (Example 1: Input Text: Is this AI? Seen on Facebook., Output: 1. 
                    Example 2: Input Text: Disney Posted this today, is it AI?, Output: 1)

                    Category 2. Text content involves getting help to recreate something.
                    (Example 1: what is this style called? What prompt should I use?, Output: 2.
                    Example 2: What would be the best model or prompts to use to get results in this style?, Output: 2)

                    Category 3. Text content does not fit category 1 or 2.
                    (Example 1: My take on the real life Simpsons, Output: 3.
                    Example 2: Why can't new buildings look more like this? Comment which apartment you want in your neighborhood. [prompts in the comments]., Output: 3)

                    Make sure to return only a single integer containing the output at the end.
                    Here is the input text that you must now classify. Input Text: {input_text}.
                    """
    
    data = query({"inputs": prompt2})

    return data[0]['generated_text'][-10:]

In [ ]:
import re   # regex

def last_number(x):
    numbers = re.findall(r'\d+', x)
    if numbers:
        return int(numbers[-1])
    else:
        return None

In [ ]:
list_of_categories = [] # initializing a list

for index, row in df.iterrows():
    x = last_number(reddit_classifier(row['text']))    # storing to 'x' the output (category)
    list_of_categories.append(x)    # adding each new output to the list 
    print(x)    # print x each time as well

In [ ]:
while len(list_of_categories) < len(df):
    list_of_categories.append(None)

df["category"] = list_of_categories

In [ ]:
df.to_csv(r'results.csv')